#Neural Nets - Final Project

##Loading Modules

In [0]:
######################################################################
# Setup python environment and change the current working directory
######################################################################
!pip install torch torchvision
!pip install quandl
!pip install Pillow==4.0.0
%mkdir -p /content/csc421/project/
%cd /content/csc421/project

    100% |████████████████████████████████| 2.0MB 10.9MB/s 
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.2.post3 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
scikit-image 0.14.2 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 6.0.0
    Uninstalling Pillow-6.0.0:
      Successfully uninstalled Pillow-6.0.0


/content/csc421/project


##Helper Functions

In [0]:
import os
import pdb
import argparse
import pickle as pkl

from collections import defaultdict

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from six.moves.urllib.request import urlretrieve
import tarfile
import pickle
import sys


def get_file(fname,
             origin,
             untar=False,
             extract=False,
             archive_format='auto',
             cache_dir='data'):
    datadir = os.path.join(cache_dir)
    if not os.path.exists(datadir):
        os.makedirs(datadir)

    if untar:
        untar_fpath = os.path.join(datadir, fname)
        fpath = untar_fpath + '.tar.gz'
    else:
        fpath = os.path.join(datadir, fname)
    
    print(fpath)
    if not os.path.exists(fpath):
        print('Downloading data from', origin)

        error_msg = 'URL fetch failure on {}: {} -- {}'
        try:
            try:
                urlretrieve(origin, fpath)
            except URLError as e:
                raise Exception(error_msg.format(origin, e.errno, e.reason))
            except HTTPError as e:
                raise Exception(error_msg.format(origin, e.code, e.msg))
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(fpath):
                os.remove(fpath)
            raise

    if untar:
        if not os.path.exists(untar_fpath):
            print('Extracting file.')
            with tarfile.open(fpath) as archive:
                archive.extractall(datadir)
        return untar_fpath

    if extract:
        _extract_archive(fpath, datadir, archive_format)

    return fpath

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
def to_var(tensor, cuda):
    """Wraps a Tensor in a Variable, optionally placing it on the GPU.

        Arguments:
            tensor: A Tensor object.
            cuda: A boolean flag indicating whether to use the GPU.

        Returns:
            A Variable object, on the GPU if cuda==True.
    """
    if cuda:
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)


def create_dir_if_not_exists(directory):
    """Creates a directory if it doesn't already exist.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)


def save_loss_plot(train_losses, val_losses, opts):
    """Saves a plot of the training and validation loss curves.
    """
    plt.figure()
    plt.plot(range(len(train_losses)), train_losses)
    plt.plot(range(len(val_losses)), val_losses)
    plt.title('BS={}, nhid={}'.format(opts.batch_size, opts.hidden_size), fontsize=20)
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(opts.checkpoint_path, 'loss_plot.pdf'))
    plt.close()


def checkpoint(encoder, decoder, idx_dict, opts):
    """Saves the current encoder and decoder models, along with idx_dict, which
    contains the char_to_index and index_to_char mappings, and the start_token
    and end_token values.
    """
    with open(os.path.join(opts.checkpoint_path, 'encoder.pt'), 'wb') as f:
        torch.save(encoder, f)

    with open(os.path.join(opts.checkpoint_path, 'decoder.pt'), 'wb') as f:
        torch.save(decoder, f)

    with open(os.path.join(opts.checkpoint_path, 'idx_dict.pkl'), 'wb') as f:
        pkl.dump(idx_dict, f)

##Getting TS Data from Quandl

In [0]:
######################################################################
# Download ticker list from Github
######################################################################
#data_fpath = get_file(fname='SP500.csv', 
#                         origin='https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP500.csv', 
#                         untar=False)
#data_fpath = get_file(fname='SP400.csv', 
#                         origin='https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP400.csv', 
#                         untar=False)

data/SP500.csv


In [0]:
import torch, re,os
from datetime import datetime as dt
from datetime import timedelta as td
from torch.autograd import Variable
import matplotlib.pyplot as plt
import quandl as qdl
import numpy as np
import pandas as pd
from os.path import abspath
from inspect import getsourcefile
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import CustomBusinessDay

def file_iterator(rootdir):
    files = []
    f = []
    g = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file[-3:] == "csv":
                f.append(os.path.join(subdir, file))
                g.append(file[:-4])
    return(f, g)
  
  
class data:
    def __init__(self, quandl_key, data_type = ["SP400"], s_d = dt.today(), e_d = dt(2018, 3, 27, 0, 0), from_end =  3*365):
        
        self.qdl_key = quandl_key
        self.end_date = e_d
        self.data_type = data_type

        if s_d < e_d :
            self.start_date = s_d
        else:
            self.start_date = self.end_date - td(days = from_end)
        
    def get_data_daily_rand(self , num_use = 10000, path_use = "",column_name = "Ticker"):
      #column_name = "ACT Symbol" for SP500
      #column_name = "Ticker" for SP400
        
        qdl.ApiConfig.api_key = self.qdl_key
        
        if path_use == "":
            pl = abspath('/content/csc421/project/data')+'/'
        else:
            pl = path_use

        cd = re.split(r'\<', pl)[0]
        t = 0
        splitty = 0
        f_it = file_iterator(cd)
        for j in f_it[0]:
            if f_it[1][splitty] in self.data_type:
                tickers = pd.read_csv(j)[column_name]
                num_samp = min(len(tickers), num_use)
                tickers = tickers[np.random.choice(len(tickers),num_samp, replace = False)]
                tickers = tickers
                for i in tickers:
                    t += 1
                    beg_d = self.start_date.strftime("%Y-%m-%d")
                    end_d = self.end_date.strftime("%Y-%m-%d")
                    if t == 1 :
                        data_use = qdl.get_table("WIKI/PRICES",
                                   qopts={"columns":["date","ticker","open","low","high","close","adj_close","volume"]},
                                   ticker= i, 
                                   date = {'gte': beg_d,'lte' : end_d})
                        data_use = data_use.set_index(["date"], drop=True)    

                    else:
                        temp = qdl.get_table("WIKI/PRICES",
                                   qopts={"columns":["date","ticker","open","low","high","close","adj_close","volume"]},
                                   ticker= i, 
                                   date = {'gte': beg_d,'lte' : end_d})
                        temp = temp.set_index(["date"], drop=True)

                        tot = [data_use, temp]
                        data_use = pd.concat(tot)
                
                if (t/num_samp *100 % 20) == 0:
                    print(t/len(tickers))
        
            splitty += 1
            print(splitty/len(f_it[0]))
        self.data_got_rand = data_use 
        
    def get_data_daily_specific(self, ticker_use):
      
        self.ticker_main = ticker_use
        self.ticker_main = ticker_use
        qdl.ApiConfig.api_key = self.qdl_key

        beg_d = self.start_date.strftime("%Y-%m-%d")
        end_d = self.end_date.strftime("%Y-%m-%d")

        data_use = qdl.get_table("WIKI/PRICES",
                                   qopts={"columns":["date","ticker","open","low","high","close","adj_close","volume"]},
                                    ticker= self.ticker_main, 
                                   date = {'gte': beg_d,'lte' : end_d})
        self.data_got_specific = data_use.set_index(["date"], drop=True)
        
        
    def set_up_features(self, which_data = "rand",ratio = [60,30,10],num_features = [30], grouping_feature = ["ticker"],
                        TS_feature_set = ["close","open","low","high"], cat_feature = [], order = []):
        """
        
        Will need to add catigorcial feature later for improved implementation
        """
        
        if not (sum(ratio) == 100):
            return "ratio entered is invalid"
        
        if which_data == "rand":
            training_d = self.data_got_rand
        
        elif which_data == "specific":
            training_d = self.data_got_specific
            
        training_d = training_d[TS_feature_set + grouping_feature]
        dd = training_d.groupby(grouping_feature)
        
        
        def make_d(list_x):
            b = list_x[0]
            for c in list_x[1:]:
                b = pd.concat([b.reset_index(), c.reset_index()], 
                              axis =1 ).drop('date', axis = 1).set_index(b.index)
            return b
        
        #unnecessary code
        #for feature_type in TS_feature_set:
        #    dd_get = {a:[group[feature_type][i : - num_features[0] + 1 + i] 
        #               for i in range(num_features[0])] for a,group in dd}    
         
        self.dd = {a:[[group[feature_type][i : - num_features[0] + 1 + i]
                       for i in range(num_features[0])]
                      for feature_type in TS_feature_set]
                   for a,group in dd} 
        #torch.t(torch.FloatTensor(S_P_Data.dd['AIG'][0][0:29]))
        dd_wanted = {}
        for a in self.dd.keys():
          w = torch.t(torch.FloatTensor(self.dd[a][0][0:29])).unsqueeze(-1)
          x = torch.t(torch.FloatTensor(self.dd[a][1][0:29])).unsqueeze(-1)
          y = torch.t(torch.FloatTensor(self.dd[a][2][0:29])).unsqueeze(-1)
          z = torch.t(torch.FloatTensor(self.dd[a][3][0:29])).unsqueeze(-1)
          aa = torch.cat((w, x, y, z), 2)
          dd_wanted[a] = aa
          #) for a in dd.keys()}
            
        self.data_cleaned = dd_wanted
        
    def as_tensor(self, value):
        torch.tensor(np.asanyarray(self.value))
        
    def get_wide(self, price_type):
        self.d_wide = data_got_rand.pivot(columns="ticker", values= price_type)
        
def save_to_pkl(path, objectname, endname):
  with open(os.path.join(path, endname), 'wb') as f:
    pkl.dump(objectname, f)
        




In [0]:
qdl_key_implament = "zFoL6yQsQfFgifpzPQFC"
S_P_Data = data(qdl_key_implament, from_end=8*365) #8*365 for 8 years back

In [0]:
S_P_Data.get_data_daily_rand(num_use = 100)
S_P_Data.set_up_features()


### Saving procured dataset as pickle
save_to_pkl('/content/csc421/project/data/', S_P_Data.data_cleaned, 'SP400.pkl')

0.5
1.0
1.0


In [0]:
### Opening pickle dataset

### if opening from Colab env:
#file_pi1 = open('/content/csc421/project/data/SP400.pkl', 'rb') 

### opening from Github:
#file_pi1 = open('/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP400.pkl', 'rb')
#SP400data = pickle.load(file_pi1)
#file_pi2 = open('/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP500.pkl', 'rb')
#SP500data = pickle.load(file_pi2)

In [0]:
######################################################################
# Download above processed Pickled data from Github
######################################################################
data_fpath = get_file(fname='SP500.pkl', 
                         origin='/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP500.pkl', 
                         untar=False)
data_fpath = get_file(fname='SP400.pkl', 
                         origin='/https://raw.githubusercontent.com/ssunger/DL4TSD/master/Data%20Sets/Constituent%20Data/SP400.pkl', 
                         untar=False)

data/SP500.pkl
data/SP400.pkl


In [0]:
### Tickers for which we have extracted data
S_P_Data.data_cleaned.keys()

torch.Size([222, 29, 4])

#Training

In [0]:
#TEST_SENTENCE = 'the air conditioning is working'

import time
start = time.time()

args = AttrDict()
args_dict = {
              'cuda':True, 
              'nepochs':100, 
              'checkpoint_dir':"checkpoints", 
              'learning_rate':0.005, 
              'lr_decay':0.99,
              'batch_size':64, 
              'hidden_size':20, 
              'decoder_type': 'rnn', 
              'attention_type': '', 
}
args.update(args_dict)

print_opts(args)
rnn_encoder, rnn_decoder = train(args)

#translated = translate_sentence(TEST_SENTENCE, rnn_encoder, rnn_decoder, None, args)
#print("source:\t\t{} \ntranslated:\t{}".format(TEST_SENTENCE, translated))

end = time.time()
print(end - start)